# 3장 고객의 전체 모습을 파악하는 테크닉 10

- use_log.csv : 센터의 이용 이력 데이터. 2018.4~2019.3 -> uselog
- customer_master.csv: 2019년 3월 말 회원 데이터 -> customer
- class_master.csv: 회원 구분 데이터(종일, 주간, 야간) -> class_master
- campaign_master.csv: 행사 구분 데이터(입회비 유무 등) -> campaign_master

## 21. 데이터를 읽어서 파악하자.

In [ ]:
import pandas as pd

uselog = pd.read_csv('use_log.csv')
print(len(uselog))
uselog.head()

197428


,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [ ]:
customer = pd.read_csv('customer_master.csv')
print(len(customer))
customer.head()

4192


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


In [ ]:
class_master = pd.read_csv('class_master.csv')
print(len(class_master))
class_master.head()

3


,class,class_name,price
0,C01,0_종일,10500
1,C02,1_주간,7500
2,C03,2_야간,6000


In [ ]:
campaign_master = pd.read_csv('campaign_master.csv')
print(len(campaign_master))
campaign_master.head()

3


,campaign_id,campaign_name
0,CA1,2_일반
1,CA2,0_입회비반액할인
2,CA3,1_입회비무료


- 기준 데이터: 고객 데이터(customer), 이용 이력 데이터(uselog)

## 22. 고객 데이터를 가공하자

In [ ]:
# customer_join = customer + class_master(join) + campaign_master(join)
customer_join = pd.merge(customer, class_master, on="class", how="left")
customer_join = pd.merge(customer_join, campaign_master, on="campaign_id", how="left")
customer_join.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,2_야간,6000,2_일반


In [ ]:
# join 전과 후 변화 없음
print(len(customer))
print(len(customer_join))

4192
4192


In [ ]:
# 결측치 확인
customer_join.isnull().sum()

,0
customer_id,0
name,0
class,0
gender,0
start_date,0
end_date,2842
campaign_id,0
is_deleted,0
class_name,0
price,0


## 23. 고객 데이터를 집계하자

In [ ]:
customer_join.groupby("class_name").count()["customer_id"]

,customer_id
class_name,
0_종일,2045
1_주간,1019
2_야간,1128


In [ ]:
customer_join.groupby("campaign_name").count()["customer_id"]

,customer_id
campaign_name,
0_입회비반액할인,650
1_입회비무료,492
2_일반,3050


In [ ]:
customer_join.groupby("gender").count()["customer_id"]

,customer_id
gender,
F,1983
M,2209


In [ ]:
customer_join.groupby("is_deleted").count()["customer_id"]

,customer_id
is_deleted,
0,2842
1,1350


- class: 종일(절반) > 야간 > 주간
- campaign: 일반 입회 > 입회 캠페인(20%)
- F/M: m > f
- 탈퇴: 1350 / 가입: 2842

In [ ]:
# 2018.4~2019.3 가입인원 집계
customer_join["start_date"]=pd.to_datetime(customer_join["start_date"])
customer_start=customer_join.loc[customer_join["start_date"]>pd.to_datetime("20180401")]
print(len(customer_start))

1361


## 24. 최신 고객 데이터를 집계하자

In [ ]:
# 가장 최근 월(2019.3)의 고객 추출 -> customer_newer
customer_join["end_date"]=pd.to_datetime(customer_join["end_date"])
customer_newer = customer_join.loc[(customer_join["end_date"]>=pd.to_datetime("20190331"))|(customer_join["end_date"].isna())]
print(len(customer_newer))
customer_newer["end_date"].unique()

# DatetimeArray: NaT(datatime형의 결측치, 탈퇴하지 않은 고객), datetime64[ns]형

2953


<DatetimeArray>
['NaT', '2019-03-31 00:00:00']
Length: 2, dtype: datetime64[ns]

- 회원 구분, 캠페인 구분, 성별로 최신 고객 데이터 전체 파악

In [ ]:
#class
customer_newer.groupby("class_name").count()["customer_id"]

,customer_id
class_name,
0_종일,1444
1_주간,696
2_야간,813


In [ ]:
#campaign
customer_newer.groupby("campaign_name").count()["customer_id"]

,customer_id
campaign_name,
0_입회비반액할인,311
1_입회비무료,242
2_일반,2400


In [ ]:
#f/m
customer_newer.groupby("gender").count()["customer_id"]

,customer_id
gender,
F,1400
M,1553


- 고객 데이터 분석 결과
1. 회원이나 성결 구분은 큰 변화가 없다.
2. 입회 캠페인은 회원 비율 변화에 영향을 미친다.
3. 이용 이력 데이터로 고객 파악을 해보자.

## 25. 이용 이력 데이터를 집계하자

In [ ]:
# uselog -> uselog_month
uselog["usedate"] = pd.to_datetime(uselog["usedate"])
uselog["연월"] = uselog["usedate"].dt.strftime("%Y%m")    #"uselog" -> "연월"(연월형태)
uselog_months = uselog.groupby(["연월", "customer_id"], as_index=False).count()   #월별("연월"), 고객별("customer_id") (->열) 그룹화  #나머지 열의 값들: 그룹별로 NaN 아닌 값들 count한 수
uselog_months.rename(columns={"log_id":"count"}, inplace=True)    #"log_id" -> "count"
del uselog_months["usedate"]    #"usedate"열 제거

uselog_months.head()

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [ ]:
# 고객별로 평균값, 중앙값, 최댓값, 최솟값 집계: uselog_customer
uselog_customer = uselog_months.groupby("customer_id").agg(["mean", "median", "max", "min" ])["count"]
uselog_customer = uselog_customer.reset_index(drop=False)
uselog_customer.head()

TypeError: agg function failed [how->mean,dtype->object]

## 26. 이용 이력 데이터로부터 정기 이용 플래그를 작성하자

- 월별 정기 이용 고객: 월/요일별 집계했을 때 최댓값 >= 4 요일이 하나라도 있으면 플래그를 1로 처리

In [ ]:
# 1. 월/요일별로 집계: uselog_weekday
uselog["weekday"] = uselog["usedate"].dt.weekday    #월~일 -> 0~6
uselog_weekday = uselog.groupby(["customer_id", "연월", "weekday"], as_index = False).count()[["customer_id", "연월", "weekday", "log_id"]]
uselog_weekday.rename(columns={"log_id":"count"}, inplace=True)
uselog_weekday.head()

,customer_id,연월,weekday,count
0,AS002855,201804,5,4
1,AS002855,201805,2,1
2,AS002855,201805,5,4
3,AS002855,201806,5,5
4,AS002855,201807,1,1


In [ ]:
# 2. 고객별로 최댓값이 4 이상이면 플래그 지정
uselog_weekday = uselog_weekday.groupby("customer_id", as_index=False).max()[["customer_id", "count"]]    #고객 단위로 집계하고 최댓값(어떤 달의 특정 요일에 정기적 방문한 횟수)
uselog_weekday["routine_flg"] = 0
uselog_weekday["routine_flg"] = uselog_weekday["routine_flg"].where(uselog_weekday["count"]<4,1)    #횟수가 4이상이면 1
uselog_weekday.head()

,customer_id,count,routine_flg
0,AS002855,5,1
1,AS008805,4,1
2,AS009013,2,0
3,AS009373,5,1
4,AS015233,5,1


## 27. 고객 데이터와 이용 이력 데이터를 결합하자

In [ ]:
# customer_join + uselog_customer + uselog_weekday
customer_join = pd.merge(customer_join, uselog_customer, on = "customer_id", how="left")
customer_join = pd.merge(customer_join, uselog_weekday[["customer_id", "routine_flg"]], on = "customer_id", how = "left")
customer_join.head()

NameError: name 'uselog_customer' is not defined

In [ ]:
customer_join.isnull().sum()

## 28. 회원 기간을 계산하자

In [ ]:
from dateutil.relativedelta import relativedelta    #날짜 차이 계산 relativedelta library import

#calc_date 열 추가
customer_join["calc_date"] = customer_join["end_date"]
customer_join["calc_date"] = customer_join["calc_date"].fillna(pd.to_datetime("20190430"))    #결측치에 20190430 대입

customer_join["membership_period"] = 0    #초기화

for i in range(len(customer_join)):
  delta = relativedelta(customer_join["calc_date"].iloc[i], customer_join["start_date"].iloc[i])
  customer_join["membership_period"].iloc[i] = delta.years*12 + delta.months    #가입 기간을 개월 수로 변환

customer_join.head()

## 29. 고객 행동의 각종 통계량을 파악하자

In [ ]:
customer_join[["mean", "median", "max", "min"]].describe()

In [ ]:
customer_join.groupby("routine_flg").count()["customer_id"]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(customer_join["membership_period"])

## 30. 탈퇴 회원과 지속 회원의 차이를 파악하자

In [ ]:
customer_end = customer_join.loc[customer_join["is_deleted"]==1]
customer_end.describe()

In [ ]:
customer_stay=customer_join.loc[customer_join["is_deleted"]==0]
customer_stay.describe()

In [ ]:
customer_join.to_csv("customer_join.csv", index=False)